In [2]:
import requests
import xmltodict
import urllib, urllib.request
import mysql.connector
import time
from IPython.display import clear_output
import uuid
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import re
#import googlemaps
from difflib import SequenceMatcher
import unicodedata
from collections import Counter
import random
from web_crawler import Crawler
import selenium as se
import mysql.connector

# Database Connection

In [4]:
# Establish database connection
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                              host='127.0.0.1',
                              database='computervision')
cursor = cnx.cursor()

DatabaseError: 2003 (HY000): Can't connect to MySQL server on '127.0.0.1:3306' (111)

In [5]:
# Semantic Scholar API Key
headers = {"x-api-key": ""}

In [10]:
# Semantic Scholar API methods
def semantic_paper_query(text):
    link = f"https://api.semanticscholar.org/graph/v1/paper/search?query={text}"
    response = requests.get(link, headers=headers).json()
    return response

def semantic_paper_details(paper_id):
    paper_link = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=externalIds,url,year,referenceCount,citationCount,publicationTypes,journal,authors,title,references.url,references.title,references.year,references.referenceCount,references.citationCount,references.authors,references.journal,references.publicationTypes,references.externalIds,authors.name,authors.affiliations,authors.paperCount,authors.hIndex,authors.aliases,embedding"
    response = requests.get(paper_link, headers=headers).json()
    return response

def semantic_author_details(author_ids):
    author_link = f"https://api.semanticscholar.org/graph/v1/author/{author_ids}?fields=affiliations,name,paperCount,hIndex,aliases"
    response = requests.get(author_link, headers=headers).json()
    return response

# Fetch all the papers in the category cs.CV from {Year_start} to {Year_end}

In [6]:
YEAR_START = 2022
YEAR_END = 2022
STEP_SIZE = 10
CATEGORY = "cs.CV"

In [28]:
# Fetch arxiv entries of type {category} with batch size {step_size}
def fetch_arxiv_entries(year_temp, current_index, STEP_SIZE):
    while True:
        try:
            arxiv_query = f'https://export.arxiv.org/api/query?search_query=cat:cs.CV+AND+submittedDate:[{year_temp}01010630+TO+{year_temp}12311645]&sortBy=submittedDate&start={current_index}&max_results={STEP_SIZE}'
            data = urllib.request.urlopen(arxiv_query)
            data_xml = data.read().decode('utf-8')
            data_dict = xmltodict.parse(data_xml)

            total_entries = int(data_dict['feed']['opensearch:totalResults']['#text'])
            print(f"{current_index}/{total_entries}")
            return data_dict['feed']['entry'], total_entries
        except KeyError:
            print("sleeping")
            time.sleep(3)

# store author information and map to paperID
def store_authors(author_information, paperID):
    if not author_information:
        try:
            # in case there is no author entry for a paper
            cursor.execute("INSERT INTO authoredBy VALUES (%s,%s)", [paperID, "0"])
        except mysql.connector.IntegrityError as err:
            pass
        return
    for author in author_information:
        try:
            name = author["name"]
        except:
            print(author)
            raise KeyError

        # Get full name instead of abbreviation
        if "." in name and author["aliases"]:
            for alias in author["aliases"]:
                if len(alias) > len(name):
                    name = alias
                if "." not in alias and len(alias.split(" ")[0]) > 2:
                    name = alias
                    break

        params = (author["authorId"], name, author["paperCount"], author["hIndex"])
        insert_author = "INSERT INTO Authors(AuthorID, Name, PaperCount, hIndex) VALUES (%s,%s,%s,%s)"
        insert_authoredBy = "INSERT INTO authoredBy VALUES (%s,%s)"

        try: # Insert author
            cursor.execute(insert_author, params)
        except mysql.connector.IntegrityError as err:
            pass # Author exists already
        try:
            cursor.execute(insert_authoredBy, (paperID, author["authorId"])) # Insert relation to paper
        except mysql.connector.IntegrityError as err:
            pass

# Store paper information
def store_paper(archive_url, paper_response, isLeaf):
    isConference = False
    isJournalArticle = False
    isReview = False
    journal = ''

    if not paper_response['paperId']:
        cursor.execute("SELECT * FROM Papers p WHERE Title=%s", [paper_response["title"]])
        duplicate_paper = cursor.fetchall()
        if duplicate_paper:
            paper_response['paperId'] = duplicate_paper[0][0]
        else:
            paper_response['paperId'] = '11111' + uuid.uuid4().hex
    if paper_response["publicationTypes"]:
        if "JournalArticle" in paper_response["publicationTypes"]:
            isJournalArticle = True
        if "Conference" in paper_response["publicationTypes"]:
            isConference = True
        if "Review" in paper_response["publicationTypes"]:
            isReview = True
    if paper_response["journal"]:
        if "name" in paper_response["journal"]:
            journal = paper_response["journal"]["name"]

    doi = "NA"
    # store DOI
    if paper_response['externalIds'] and "DOI" in paper_response['externalIds']:
        doi = paper_response["externalIds"]["DOI"]
    params = [paper_response['paperId'], paper_response['title'], archive_url, paper_response['year'], CATEGORY, isReview, isConference, isJournalArticle,paper_response['referenceCount'], paper_response['citationCount'], journal, isLeaf, paper_response['url'], doi, None, 0]
    insert_paper = "INSERT INTO Papers VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    try:
        cursor.execute(insert_paper, params)
        return True
    except mysql.connector.IntegrityError as err:
        # If paper already exists check
        if isLeaf:
            return False

        check_if_leaf = "SELECT Leaf FROM Papers WHERE PaperID = %s"
        cursor.execute(check_if_leaf, [paper_response['paperId']])
        isLeafPaper = cursor.fetchone()[0]
        if isLeafPaper == isLeaf:
            return False

        update_leaf = "UPDATE Papers SET Leaf = %s WHERE PaperID = %s"
        cursor.execute(update_leaf, (isLeaf,paper_response['paperId']))
        return True

    except KeyError as err:
        print(err)
        raise KeyError

# Store reference information in referencedBy table
def store_reference(paperId, referenced_by):
    try:
        insert_reference = "INSERT INTO referencedBy VALUES (%s,%s)"
        cursor.execute(insert_reference, (paperId, referenced_by))
    except mysql.connector.IntegrityError as err:
        pass

In [11]:
for i in range(YEAR_END - YEAR_START + 1):
    year_temp = YEAR_START + i
    print(year_temp)
    current_index = 0
    var = 0

    while True:
        # Get title of all entries in a year from arxiv
        entries, total_entries = fetch_arxiv_entries(year_temp, current_index, STEP_SIZE)
        current_index += STEP_SIZE

        # Fetch information for papers from semantic scholar
        for entry in entries:
            var += 1
            print(var)
            archive_link = entry["id"]
            entry_title = entry["title"].replace("-", " ").replace("\n", "")
            while True:
                try:
                    response = semantic_paper_query(entry_title)

                    if not response["total"]:
                        break
                    else:
                            # Store papers' information in database
                            paperId = response["data"][0]["paperId"]
                            cursor.execute("select * from Papers p where p.PaperID = %s",[paperId])
                            exist_paper = cursor.fetchall()
                            if exist_paper and not exist_paper[0][-5]:
                                print("exists")
                                break
                            paper_response = semantic_paper_details(paperId)
                            if paper_response["referenceCount"] > 70 or paper_response["referenceCount"] == 0 or len(paper_response["authors"]) > 10:
                                print("too big")
                                break
                            success = store_paper(archive_link, paper_response, False)

                            if not success:
                                print("no success")
                                break

                            store_authors(paper_response['authors'], paperId)
                            # Store references' information in database
                            for reference in paper_response["references"]:
                                store_paper(None, reference, True)
                                store_reference(reference["paperId"], paperId)
                                if reference["authors"]:
                                    for author in reference["authors"]:
                                        if not author["authorId"]:
                                            continue
                                        author_information = semantic_author_details(author["authorId"])
                                        store_authors([author_information], reference["paperId"])
                                else:
                                    store_authors(None, reference["paperId"])
                    break
                except KeyError:
                    continue
                except mysql.connector.DataError as e:
                    break

            cnx.commit()

        if current_index > total_entries:
            break
    clear_output(wait=True)

2022
18730/19382
sleeping
18730/19382
1
2
3
4
5
too big
6
too big
7
8
9
10
18740/19382
11
12
too big
13
14
15
16
17
18
19
too big
20
18750/19382
sleeping
18750/19382
sleeping
18750/19382
21
too big
22
23
24
25
26
too big
27
28
29
too big
30
18760/19382
sleeping
18760/19382
31
32
33
too big
34
35
36
too big
37
38
39
40
too big
18770/19382
41
42
too big
43
44
45
46
47
48
49
50
18780/19382
sleeping
18780/19382
sleeping
18780/19382
sleeping
18780/19382
sleeping
18780/19382
51
52
53
54
too big
55
56
57
58
59
60
18790/19382
sleeping
18790/19382
sleeping
18790/19382
sleeping
18790/19382
sleeping
18790/19382
sleeping
18790/19382
sleeping
18790/19382
61
62
too big
63
64
65
66
67
68
69
70
18800/19382
71
too big
72
73
74
75
76
too big
77
78
79
80
18810/19382
sleeping
18810/19382
81
82
83
too big
84
85
86
87
88
89
90
18820/19382
sleeping
18820/19382
sleeping
18820/19382
sleeping
18820/19382
91
92
93
94
95
96
97
98
99
100
18830/19382
sleeping
18830/19382
101
too big
102
103
104
too big
105
106
107


# Fetch Author information

In [42]:
# In case this step was left out before due to api constraints
cursor.execute("select a.AuthorID from Authors a where a.hIndex is NULL and a.AuthorID != 0")
authorIDs = [x[0] for x in cursor.fetchall()]
start_index = 0
for idx, authorID in enumerate(authorIDs):
    if idx % 10 == 0:
        print(idx)
    author = semantic_author_details(authorID)
    print(author)
    try:
        name = author["name"]
        if "." in name and author["aliases"]:
            for alias in author["aliases"]:
                if len(alias) > len(name):
                    name = alias
                if "." not in alias and len(alias.split(" ")[0]) > 2:
                    name = alias
                    break

        params = (author["authorId"], name, author["paperCount"], author["hIndex"])
        insert_author = "UPDATE Authors a SET a.Name = %s, a.PaperCount = %s, hIndex = %s where a.AuthorID = %s"
        cursor.execute(insert_author, (name,author["paperCount"],author["hIndex"],authorID))
        cnx.commit()
    except KeyError: # Delete if the entries in the semantic scholar api do not exist anymore
        cursor.execute("Delete from authoredBy a where a.AuthoredByID = %s", [authorID])
        cursor.execute("Delete from Authors a where a.AuthorID = %s", [authorID])
        continue

0
{'error': 'Author with id 1797153 not found'}
{'error': 'Author with id 2113556648 not found'}
{'error': 'Author with id 2118824286 not found'}
{'error': 'Author with id 2132003885 not found'}
{'error': 'Author with id 2135474727 not found'}
{'error': 'Author with id 2142417439 not found'}
{'error': 'Author with id 2146045574 not found'}
{'error': 'Author with id 2146282560 not found'}
{'error': 'Author with id 2154548819 not found'}
{'error': 'Author with id 2154554837 not found'}
10
{'error': 'Author with id 2157015792 not found'}
{'error': 'Author with id 2157049095 not found'}
{'error': 'Author with id 2157050777 not found'}
{'error': 'Author with id 2157051527 not found'}
{'error': 'Author with id 2158166147 not found'}
{'error': 'Author with id 2171014391 not found'}
{'error': 'Author with id 2180392892 not found'}
{'error': 'Author with id 2180415436 not found'}
{'error': 'Author with id 2180415447 not found'}
{'error': 'Author with id 2180509532 not found'}
20
{'error': 'Auth

# Add Publisher based on the DOI

In [ ]:
cursor.execute("select distinct p.PaperID, p.Title, GROUP_CONCAT(b.AuthoredByID, ':',  a.Name), p.DOI, count(*) from Papers p, authoredBy b, Authors a where p.PaperID not like '1111%' and p.DOI like '%/%' and p.PaperID = b.PaperID and a.AuthorID = b.AuthoredByID group by p.PaperID, p.Title, p.DOI")
left_papers = cursor.fetchall()

In [ ]:
all_prefixes = [paper[3].split("/")[0] for paper in left_papers]
all_prefix = list(set(all_prefixes))

In [ ]:
journal_prefix = {}

for idx, prefix in enumerate(all_prefix):
    if idx % 10 == 0:
        print(idx)
    try:
        resp = requests.get(f"https://api.crossref.org/prefixes/{prefix}").json()
        journal_prefix[prefix] = resp["message"]["name"]
    except ValueError:
        journal_prefix[prefix] = None

In [ ]:
grouped_journals = {}
grouped_journal_ids = {}
x = Counter(all_prefixes)
for k, v in x.items():
    if journal_prefix[k] not in grouped_journals:
        grouped_journals[journal_prefix[k]] = v
        grouped_journal_ids[journal_prefix[k]] = [k]
    else:
        grouped_journals[journal_prefix[k]] += v
        grouped_journal_ids[journal_prefix[k]].append(k)

In [ ]:
for publisher in grouped_journals:
    prefixes = grouped_journal_ids[publisher]
    for prefix in prefixes:
        cursor.execute("update Papers p set p.publisher = %s where p.DOI like %s",[publisher,prefix + "/%"])
        cnx.commit()

# Fetch Affiliations

In [5]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def preprocess_author_name(name):
    return unicodedata.normalize('NFD', name.lower().replace("ü", "ue").replace("ß", "ss").replace("ä","ae").replace("ö", "oe")).encode('ascii', 'ignore').decode("utf-8") .replace("-", " ")

In [42]:
train_papers = []
with open(r'./test_papers.txt', 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]
        # add current item to the list
        train_papers.append(x)
in_params = ','.join(['%s'] * len(train_papers))

In [82]:
cursor.execute("with x as (select p.PaperID from Papers p where p.PaperID in (%s))"
               "select p.`primary author` from Papers p where p.PaperID in (select PaperID from x) and p.`primary author` not in (select AuthorID from affiliatedTo) "
               "UNION select p2.`primary author` from Papers p, referencedBy b, Papers p2 where p.PaperID in (select PaperID from x) and b.ReferencedByID = p.PaperID "
               "and b.ReferenceID = p2.PaperID and p2.`primary author` not in (select AuthorID from affiliatedTo)" % in_params, train_papers)
authors_to_fetch = [x[0] for x in cursor.fetchall()]
in_paramss = ','.join(['%s'] * len(authors_to_fetch))
cursor.execute("select p.PaperID, p.DOI from Papers p, authoredBy b where p.PaperID = b.PaperID and b.AuthoredByID in (%s) and p.publisher = 'Wiley' group by p.PaperID" % in_paramss, authors_to_fetch)
cat_papers = cursor.fetchall()
len(cat_papers)

124

In [58]:
cursor.execute('select p2.publisher, count(*) from Papers p, referencedBy b, Papers p2 where p.PaperID in (%s) and p.PaperID = b.ReferencedByID and b.ReferenceID = p2.PaperID and p2.`primary author` not in (select a.AuthorID from affiliatedTo a) group by p2.publisher order by count(*) ASC' % params, train_papers)
cursor.fetchall()

[('American Association for Clinical Chemistry (AACC)', 1),
 ('RCN Publishing Ltd.', 1),
 ('Cognitive Computational Neuroscience', 1),
 ('International Glaciological Society', 1),
 ('Verlag der Technischen Universitat Graz', 1),
 ('Mary Ann Liebert Inc', 1),
 ('Harvard Education Publishing Group', 1),
 ('Bellwether  Publishing', 1),
 ('Society of Automotive Engineers of Japan, Inc.', 1),
 ('Associated Professional Sleep Societies (APSS)', 1),
 ('Fuji Technology Press Ltd.', 1),
 ('Centers for Disease Control and Prevention (CDC)', 1),
 ('AVES YAYINCILIK A.Ş.', 1),
 ('IBM', 1),
 ('American Academy of Insurance Medicine', 1),
 ('Japan Society for Fuzzy Theory and Intelligent Informatics', 1),
 ('International Journal of Science and Research', 1),
 ('American Society of Clinical Oncology (ASCO)', 1),
 ('European Open Science Publishing', 1),
 ('Health Affairs (Project Hope)', 1),
 ('Resilience Alliance, Inc.', 1),
 ('Korean Society of Magnetic Resonance in Medicine', 1),
 ('Irish College 

In [27]:
train_papers = pd.read_csv("train_set.csv")["PaperID"].to_list()
params = ','.join(['%s'] * len(train_papers))

In [59]:
cursor.execute('select distinct p2.PaperID, p2.DOI from Papers p, referencedBy b, Papers p2, authoredBy b2 where p.PaperID in (%s) and p.PaperID = b.ReferencedByID and b.ReferenceID = p2.PaperID and p2.publisher = "Association for Computing Machinery (ACM)" and p2.PaperID = b2.PaperID and b2.AuthoredByID not in (select a.AuthorID from affiliatedTo a) group by p2.PaperID, p2.DOI order by p2.PaperID ASC' % params, train_papers)
cat_papers = cursor.fetchall()

In [60]:
print(len(cat_papers))
cat_papers

361


[('0024d00985ed1e64b274cec37dbfc81f03a23d1b', '10.1145/3414685.3417808'),
 ('00734497ceb0889f2f52cb9bb721f62a15799ebc', '10.1145/2682899'),
 ('00ac356fe8c831ebf6237a88192be1584ecd23a9', '10.1145/2461912.2461969'),
 ('00da819af2720a13c4bd80a9eba1244fa2896efc', '10.1145/288216.288266'),
 ('01040a9b6e05300f9d1f83737d2f2769ae4278dd', '10.1145/2601097.2601160'),
 ('0118ac0658f4bf765a0545bfa8769c2fc369cf3f', '10.1145/2964284.2964322'),
 ('01a3b98120cb6491e822602f01b005db8829982d', '10.1145/3510540'),
 ('036277d492dd5777e87e5b33ffd809e5c617a37a', '10.1145/2307636.2307658'),
 ('0372728b9a2def008ef3240a62362f0afbfb5d43', '10.1145/1401890.1401946'),
 ('0373442fb84f7783f3453c46f8cccc12d184dbbc', '10.1145/2647868.2654970'),
 ('03c985d681c7b5ba7fe217aa49a365a40e7ea47e', '10.1145/311535.311586'),
 ('04190a0c5b5f605fa734cc038d86acae3ef23ab8', '10.1145/3072959.3073693'),
 ('05b90ce80a2ee5e6a6126d24b29bdfeceaef6a0e', '10.1145/2393347.2393364'),
 ('0617b5e7921351c6126bece9e3a9dc53c084bd14', '10.1145/339

In [85]:
crawler = Crawler()

In [86]:
class Found(Exception): pass
for idx, paper in enumerate(cat_papers):

    if idx % 10 == 0:
        print(idx)

    cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID not in (SELECT a.AuthorID from affiliatedTo a) and b.AuthoredByID = a.AuthorID",[paper[0]])
    authors = cursor.fetchall()

    if not authors or authors[0][0] == "0":
        continue

    #affiliations = crawler.ieee_request(paper[1],scrape=True)
    #affiliations = crawler.spie_request(paper[1])
    try:
        affiliations = crawler.wiley_request(paper[1])
        #affiliations = crawler.ieee_request(paper[1],scrape=True)
        #affiliations = crawler.springer_request(paper[1], scrape=True)
        #affiliations = crawler.science_direct_scrape_request(paper[1])
        #affiliations = crawler.acm_request(paper[1])
    except (se.common.exceptions.ElementNotInteractableException, se.common.exceptions.ElementClickInterceptedException):
        continue
    #affiliations = crawler.oxford_request(paper[1])
    #affiliations = crawler.science_direct_request(paper[1], scrape=True)

    if isinstance(affiliations, str):
        print("Wrong DOI")
        continue

    if not affiliations:
        time.sleep(2 + random.random())
        continue

    for author in authors:
        try:
            print(author)
            affiliation_country = None
            found = False
            most_similar_index = -1
            try:
                author_name_tokens = preprocess_author_name(author[1]).split()
                aff_authors = [preprocess_author_name(aff_auth[0]).split() for aff_auth in affiliations]
                for author_name_token in author_name_tokens:
                    for idx, aff_author in enumerate(aff_authors):
                        if any(0.8 <= similar(author_name_token, aff) for aff in aff_author):
                            most_similar_index = idx
                            found = True
                            raise Found
            except Found:
                pass
            if not found:
                print("Not found")
                continue

            all_affiliations = [affiliations[most_similar_index][1]]

            if not all_affiliations[0].strip() or all_affiliations[0] == "None":
                continue
            for affiliation in all_affiliations:
                cursor.execute("SELECT * FROM Affiliations a WHERE a.Name=%s", [affiliation[:1028]])
                entry = cursor.fetchone()
                if not entry:
                    cursor.execute("INSERT INTO Affiliations (Name,Country) VALUES (%s,%s)", [affiliation[:1028], affiliation_country])
                    lastrowid = cursor.lastrowid
                else:
                    lastrowid = entry[0]
                cursor.execute("INSERT INTO affiliatedTo VALUES (%s,%s)", [author[0],lastrowid])
        except (AttributeError, mysql.connector.DatabaseError,  se.common.exceptions.WebDriverException, TypeError) as e:
            print(e)
            print("error")
            fill = cursor.fetchall()
            time.sleep(1 + random.random())
            continue
    time.sleep(1 + random.random())
    cnx.commit()
    print("---------")

0
('102881550', 'David G Kendall')
['Statistical Laboratory, 16 Mill Lane, Cambridge CB2 1SB, U.K.']
---------
('10406199', 'Garry Hunt')
['Meteorological Office, Bracknell']
---------
('122631505', 'Tao Su')
['Department of Electrical and Computer Engineering, University of Texas at Austin, Austin, Texas 78712-1084']
---------
('13314148', 'Daniel Nyfeler')
['Agroscope Reckenholz-Tänikon Research Station ART, Reckenholzstrasse 191, 8046 Zürich, Switzerland']
('1403749756', 'Olivier Huguenin‐elie')
['Agroscope Reckenholz-Tänikon Research Station ART, Reckenholzstrasse 191, 8046 Zürich, Switzerland']
('144112254', 'John Connolly')
['Environmental and Ecological Modelling Group, School of Mathematical Sciences, University College Dublin, Dublin 4, Ireland']
('38472264', 'Matthias Suter')
['Agroscope Reckenholz-Tänikon Research Station ART, Reckenholzstrasse 191, 8046 Zürich, Switzerland']
('4363359', 'Emmanuel Frossard')
['Institute of Plant Sciences, ETH Zürich, Eschikon 33, 8315 Lindau

# Entity Mapping

## Universities

In [7]:
uni = pd.read_csv("./EntityMapping/universities_w_aliases.csv").rename(columns={"university":"id", "alias":"final_alias"}) # all universities
uni_organizations = pd.read_csv("./EntityMapping/universities_organizations.csv") # university organizations
uni_facilities = pd.read_csv("./EntityMapping/university_facilities.csv").rename(columns={"affiliatedUniversity":"id"}) # university facilities
uni_languages = pd.read_csv("./EntityMapping/universities_language_labels.csv").rename(columns={"university": "id", "label": "alias"}) # universities in different languages

# Convert to string to aggregate it later
uni['final_alias'] = uni['final_alias'].astype(str)
uni_organizations['alias'] = uni_organizations['alias'].astype(str)
uni_facilities['alias'] = uni_facilities['alias'].astype(str)

# Preprocess and aggregate each table
uni = uni.groupby(['id', 'universityLabel'], as_index=False)["final_alias"].agg({'final_alias': '|'.join})
uni_languages = uni_languages.groupby(["id"], as_index=False)["alias"].agg({'alias': '|'.join})

uni_organizations = uni_organizations.groupby(["organizationLabel", "affiliatedUniversity"], as_index=False)["alias"].agg({'alias': '|'.join})
uni_organizations["alias"] = uni_organizations["organizationLabel"] + "|" + uni_organizations["alias"]
uni_organizations = uni_organizations.drop_duplicates("organizationLabel")
uni_organizations = uni_organizations.groupby(["affiliatedUniversity"], as_index=False)["alias"].agg({'alias': '|'.join})

uni_facilities = uni_facilities.groupby(["organizationLabel", "affiliatedUniversityLabel"], as_index=False)["alias"].agg({'alias': '|'.join})
uni_facilities = uni_facilities.drop_duplicates("organizationLabel")
uni_facilities["alias"] = uni_facilities["organizationLabel"] + "|" + uni_facilities["alias"]
uni_facilities = uni_facilities.groupby(["affiliatedUniversityLabel"], as_index=False)["alias"].agg({'alias': '|'.join})

# Merge all aliases into one table
temp = pd.merge(uni, uni_organizations, left_on="id", right_on="affiliatedUniversity", how="left")
temp = pd.merge(temp, uni_facilities, left_on="universityLabel", right_on="affiliatedUniversityLabel", how="left")
temp = temp.fillna("nan")
temp["final_alias"] = temp["final_alias"] + "|" + temp["alias_x"] + "|" + temp["alias_y"]
# Clear Duplicates and nan
all_universities_list = []
for i, row in temp.iterrows():
    cleared_aliases = list(set(filter(lambda x: x != "nan", row[2].split("|"))))
    for alias in cleared_aliases:
        all_universities_list.append((row[1], alias))
    all_universities_list.append((row[1], row[1]))
all_universities = pd.DataFrame(all_universities_list, columns=["Label", "Alias"]).drop_duplicates()
all_universities = all_universities[all_universities["Alias"].str.len() > 2].drop_duplicates("Alias",keep=False)

In [9]:
exclude_labels = {}
exclude_alias = []
for idx, row in all_universities.iterrows():
    if idx % 1000 == 0:
        print(idx)
    aliases = all_universities.loc[(all_universities["Alias"].str.contains(row[1], regex=False)) & (row[0] != all_universities["Label"])]
    valid_aliases = aliases["Alias"].to_list()
    valid_aliases = list(filter(lambda x: x!=row[1],valid_aliases))
    if valid_aliases:
        exclude_alias.append(row[1])
        exclude_labels[row[1]] = aliases["Alias"].to_list()

0
1000
4000
5000
6000
7000
8000
9000
10000
12000
13000
14000
15000
16000
17000
18000
19000
23000
25000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
84000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000


In [13]:
all_universities = all_universities.groupby("Label", as_index=False)["Alias"].agg({"Alias": "|".join})
all_universities

,Label,Alias
0,"""Alexandru cel Bun"" Military Academy",Academia Militară a Forțelor Armate „Alexandru...
1,"""Andrei Saguna"" University in Constanta",Universitatea ANDREI SAGUNA Constanta|Universi...
2,"""Aurel Vlaicu"" University of Arad",Université Aurel Vlaicu d’Arad|Universitatea „...
3,"""Drăgan"" European University of Lugoj",Universitatea Europeană Drăgan|Universitatea E...
4,"""King Ferdinand I"" University",Universitatea Regele Ferdinand I|I. Ferdinánd ...
...,...,...
18469,ŠKODA AUTO University,ŠKODA AUTO Vysoká škola o.p.s.|ŠKODA AUTO VYSO...
18470,Šiauliai University,Universität Siauliai|Šiaulių pedagoginis insti...
18471,Ștefan cel Mare University of Suceava,université de Suceava|Stefan cel Mare Universi...
18472,日本東京都興亞醫學館,日本東京都興亞醫學館


## Companies

In [56]:
# Fetch Data
all_companies = pd.read_csv("./EntityMapping/companies_enterprises.csv")
companies_aliases = pd.read_csv("./EntityMapping/companies_aliases.csv")

companies_w_sub = pd.read_csv("./EntityMapping/companies_w_subsidiaries.csv")
companies_w_sub = companies_w_sub[companies_w_sub["companyLabel"] != companies_w_sub["subsidiaryLabel"]]

# Aggregate
companies_aliases = companies_aliases.groupby(["companyLabel"], as_index = False)["alias"].agg({'label': '|'.join})
all_companies = pd.merge(all_companies, companies_aliases, left_on="label", right_on="companyLabel", how="left")[["label_x", "label_y"]].rename(columns={"label_x": "companyLabel", "label_y":"alias"})
all_companies = all_companies.fillna("nan")
all_companies_list = []
for i, row in all_companies.iterrows():
    cleared_aliases = list(set(filter(lambda x: x != "nan", row[1].split("|"))))
    for alias in cleared_aliases:
        all_companies_list.append((row[0], alias))
    all_companies_list.append((row[0], row[0]))
all_companies = pd.DataFrame(all_companies_list, columns=["Label","Alias"]).drop_duplicates()

In [57]:
# Map all subsidiaries to root company
parentStillExists = True

while parentStillExists:
    parentStillExists = False
    subsidiaries = companies_w_sub[companies_w_sub["companyLabel"].isin(companies_w_sub["subsidiaryLabel"])]
    for i, sub in subsidiaries.iterrows():
        parent_row = companies_w_sub[companies_w_sub["subsidiaryLabel"] == sub[0]]
        if len(parent_row) > 1:
            continue
        parent = parent_row.iloc[0][0]
        companies_w_sub.at[i, "companyLabel"] = parent
        parentStillExists = True

'\nparentStillExists = True\n\nwhile parentStillExists:\n    parentStillExists = False\n    subsidiaries = companies_w_sub[companies_w_sub["companyLabel"].isin(all_universities["Alias"])]\n    for i, sub in subsidiaries.iterrows():\n        parent_row = all_universities[all_universities["Alias"] == sub[0]]\n        parent = parent_row.iloc[0][0]\n        companies_w_sub.at[i, "companyLabel"] = parent\n        parentStillExists = True\n'

In [58]:
parent_remapping = {}
for i, values in companies_w_sub.iterrows():
    parent_remapping[values[1]] = values[0]
all_companies["Label"] = all_companies["Label"].apply(lambda x: parent_remapping[x] if x in parent_remapping else x)
all_companies = all_companies[all_companies["Alias"].str.len() > 3].drop_duplicates().drop_duplicates("Alias",keep=False)

In [43]:
exclude_labels = {}
exclude_alias = []
for idx, row in all_companies.iterrows():
    if idx % 1000 == 0:
        print(idx)
    aliases = all_companies[all_companies["Alias"].str.contains(row[1], regex=False)]
    valid_aliases = aliases["Alias"].to_list()
    valid_aliases = list(filter(lambda x: x!=row[1],valid_aliases))
    if valid_aliases:
        exclude_alias.append(row[1])
        exclude_labels[row[1]] = aliases["Alias"].to_list()
all_companies = all_companies[-all_companies["Alias"].isin(exclude_alias)]

0
1000
2000
3000
4000
6000
7000
8000
9000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000


In [59]:
all_companies = all_companies.groupby(["Label"], as_index = False)["Alias"].agg({'Alias': '|'.join})

# Aggregate Affiliations with the table

In [60]:
def fetch_all_affiliations():
    cursor.execute("select * from Affiliations a")
    all_affiliations = cursor.fetchall()
    for idx, aff_infos in enumerate(all_affiliations):
        if "Univ." in aff_infos[1] or "univ." in aff_infos[1]:
            aff = aff_infos[1].replace("Univ.", "University").replace("univ.", "University")
            all_affiliations[idx] = (aff_infos[0], aff, aff_infos[2])
    return all_affiliations

In [61]:
def check_occurrences(table):
    affiliation_occurrences = []
    cursor.execute("select * from Affiliations a")
    all_affiliation_infos = cursor.fetchall()
    match_table = []
    max_length = len(table)
    affiliations = [affiliation[1] for affiliation in all_affiliation_infos]
    for i, row in table.iterrows():
        if i % 1000 == 0:
            print(f"{i}/{max_length}")

        search_for = (row[0] + "|" + row[1]).replace("（","").replace("）","").replace("(","").replace(")","").split("|")
        c = 0
        for affiliation_info in all_affiliation_infos:
            affiliation = affiliation_info[1]
            try:
                if affiliation in search_for:
                    match_table.append((row[0], affiliation_info[0], affiliation))
                    c += 1

            except re.error:
                continue

        affiliation_occurrences.append((row[0], c))
    return sorted(affiliation_occurrences, key=lambda x: x[1], reverse=True)

In [62]:
occurences = check_occurrences(all_companies)
occurences

0/47722
1000/47722
2000/47722
3000/47722
4000/47722
5000/47722
6000/47722
7000/47722
8000/47722
9000/47722
10000/47722
11000/47722
12000/47722
13000/47722
14000/47722
15000/47722
16000/47722
17000/47722
18000/47722
19000/47722
20000/47722
21000/47722
22000/47722
23000/47722
24000/47722
25000/47722
26000/47722
27000/47722
28000/47722
29000/47722
30000/47722
31000/47722
32000/47722
33000/47722
34000/47722
35000/47722
36000/47722
37000/47722
38000/47722
39000/47722
40000/47722
41000/47722
42000/47722
43000/47722
44000/47722
45000/47722
46000/47722
47000/47722


[('AT&T', 3),
 ('Airbus Group', 3),
 ('Lenovo', 3),
 ('Maxar Technologies', 3),
 ('SoftBank Group', 3),
 ('ASUS', 2),
 ('Adobe', 2),
 ('Alibaba Group', 2),
 ('Amazon', 2),
 ('Amgen', 2),
 ('Baidu', 2),
 ('Broadcom Inc.', 2),
 ('Cisco Systems', 2),
 ('CyberAgent', 2),
 ('Dell Technologies', 2),
 ('Delphi Technologies', 2),
 ('ExxonMobil', 2),
 ('Foxconn', 2),
 ('General Electric', 2),
 ('Huawei', 2),
 ('Johnson & Johnson', 2),
 ('Koninklijke Philips NV', 2),
 ('Lockheed Martin', 2),
 ('Meta Platforms', 2),
 ('Microsoft', 2),
 ('Mozilla Corporation', 2),
 ('NetEase', 2),
 ('Samsung Electronics', 2),
 ('Schlumberger', 2),
 ('Seiko Epson', 2),
 ('Shell', 2),
 ('Swisscom', 2),
 ('TIM Group', 2),
 ('Telefónica', 2),
 ('Uber Technology', 2),
 ('Verizon', 2),
 ('Xerox', 2),
 ('iRobot', 2),
 ('3Shape', 1),
 ('Abbott Laboratories', 1),
 ('Adidas AG', 1),
 ('Agilent Technologies', 1),
 ('AkzoNobel', 1),
 ('Alphabet Inc.', 1),
 ('Altran', 1),
 ('Ambu', 1),
 ('Anglo American plc', 1),
 ('Apple', 1)

In [46]:
filtered_university = []
to_classify = sorted(list(filter(lambda x: x[1]>1 and x[0] not in filtered_university, occurences)),key=lambda x:x[1], reverse=True)
len(to_classify)

153

In [64]:
search_term_start = "Adobe"
search_term = all_companies[all_companies["Label"] == search_term_start].iloc[0][1]
current_search_term = search_term.split("|")
current_search_term.append(search_term_start)
for idx, x in enumerate(current_search_term):
    print(f"{idx}: {x}")

0: Adobe Systems
1: Adobe Inc.
2: Adobe
3: Allegorithmic
4: Adobe


In [65]:
to_be_filtered = []
matched_list = []
cursor.execute("select * from Affiliations a")
all_affiliation_infos = cursor.fetchall()
for aff in all_affiliation_infos:

    if aff[1] in current_search_term:
        matched_list.append((search_term_start, aff[0], aff[1]))
current_match = pd.DataFrame(matched_list)
current_match

,0,1,2
0,Adobe,208,Adobe
1,Adobe,103964,Adobe Inc.


In [36]:
to_filter = []
ids_to_remap = current_match[1].tolist()
ids_to_remap = list(filter(lambda x: x not in to_filter, ids_to_remap))
current_id = get_affiliation_id(search_term_start)
insert_affiliations(current_id, ids_to_remap)

In [66]:
sorted(list(filter(lambda x: x[1]>1 and x[1]<= 100, occurences)),key=lambda x:x[1])

[('ASUS', 2),
 ('Adobe', 2),
 ('Alibaba Group', 2),
 ('Amazon', 2),
 ('Amgen', 2),
 ('Baidu', 2),
 ('Broadcom Inc.', 2),
 ('Cisco Systems', 2),
 ('CyberAgent', 2),
 ('Dell Technologies', 2),
 ('Delphi Technologies', 2),
 ('ExxonMobil', 2),
 ('Foxconn', 2),
 ('General Electric', 2),
 ('Huawei', 2),
 ('Johnson & Johnson', 2),
 ('Koninklijke Philips NV', 2),
 ('Lockheed Martin', 2),
 ('Meta Platforms', 2),
 ('Microsoft', 2),
 ('Mozilla Corporation', 2),
 ('NetEase', 2),
 ('Samsung Electronics', 2),
 ('Schlumberger', 2),
 ('Seiko Epson', 2),
 ('Shell', 2),
 ('Swisscom', 2),
 ('TIM Group', 2),
 ('Telefónica', 2),
 ('Uber Technology', 2),
 ('Verizon', 2),
 ('Xerox', 2),
 ('iRobot', 2),
 ('AT&T', 3),
 ('Airbus Group', 3),
 ('Lenovo', 3),
 ('Maxar Technologies', 3),
 ('SoftBank Group', 3)]

In [67]:
to_classify = sorted(list(filter(lambda x: 1 < x[1] <= 40, occurences)), key=lambda x:x[1])
for temp_search_term in sorted(to_classify, key=lambda x: x[1], reverse=True):
    print(temp_search_term)
    regex = all_companies[all_companies["Label"] == temp_search_term[0]].iloc[0][1]
    regex = (regex + "|" + temp_search_term[0]).split("|")
    cursor.execute("select * from Affiliations")
    all_entries = cursor.fetchall()
    ids_to_replace = []
    for entry in all_entries:
        if entry[1] in regex:
            ids_to_replace.append(entry[0])
    if len(ids_to_replace) < 2:
        print("skip")
        continue
    print(temp_search_term[0])
    print(len(ids_to_replace))
    current_id = get_affiliation_id(temp_search_term[0])
    insert_affiliations(current_id,ids_to_replace)

('AT&T', 3)
AT&T
3
('Airbus Group', 3)
Airbus Group
3
('Lenovo', 3)
Lenovo
3
('Maxar Technologies', 3)
Maxar Technologies
3
('SoftBank Group', 3)
SoftBank Group
3
('ASUS', 2)
ASUS
2
('Adobe', 2)
Adobe
2
('Alibaba Group', 2)
Alibaba Group
2
('Amazon', 2)
Amazon
2
('Amgen', 2)
Amgen
2
('Baidu', 2)
Baidu
2
('Broadcom Inc.', 2)
Broadcom Inc.
2
('Cisco Systems', 2)
Cisco Systems
2
('CyberAgent', 2)
CyberAgent
2
('Dell Technologies', 2)
Dell Technologies
2
('Delphi Technologies', 2)
Delphi Technologies
2
('ExxonMobil', 2)
ExxonMobil
2
('Foxconn', 2)
Foxconn
2
('General Electric', 2)
General Electric
2
('Huawei', 2)
Huawei
2
('Johnson & Johnson', 2)
Johnson & Johnson
2
('Koninklijke Philips NV', 2)
Koninklijke Philips NV
2
('Lockheed Martin', 2)
Lockheed Martin
2
('Meta Platforms', 2)
Meta Platforms
2
('Microsoft', 2)
Microsoft
2
('Mozilla Corporation', 2)
Mozilla Corporation
2
('NetEase', 2)
NetEase
2
('Samsung Electronics', 2)
Samsung Electronics
2
('Schlumberger', 2)
Schlumberger
2
('Seiko

In [275]:
steps = 20
cursor.execute("select * from Affiliations a where a.entity_name is null")
entity_mapping = cursor.fetchall()
ids = [x[0] for x in entity_mapping]
names = [x[1] for x in entity_mapping]
len(ids)

22616

# Aggregate Affiliations

In [8]:
def get_affiliation_id(grouped_affiliation_name):
    cursor.execute("SELECT * FROM Affiliations a WHERE a.Name=%s", [grouped_affiliation_name])
    entry = cursor.fetchone()
    if not entry:
        cursor.execute("INSERT INTO Affiliations (Name) VALUES (%s)", [grouped_affiliation_name])
        affiliation_id = cursor.lastrowid
    else:
        affiliation_id = entry[0]
    cnx.commit()
    return affiliation_id

def insert_affiliations(affiliation_id, matching_affiliations):
    for affiliation in matching_affiliations:
        if affiliation == affiliation_id:
            continue
        cursor.execute("select * from affiliatedTo a where a.affiliatedTo = %s", [affiliation])
        selected_authors = cursor.fetchall()
        for selected_author in selected_authors:
            try:
                cursor.execute("INSERT INTO affiliatedTo VALUES (%s, %s)", [selected_author[0],affiliation_id])
            except mysql.connector.IntegrityError:
                continue
        cursor.execute("delete from affiliatedTo a where a.affiliatedTo = %s", [affiliation])
        cursor.execute("delete from Affiliations a where a.AffiliationID = %s", [affiliation])
    cnx.commit()

In [9]:
cursor.execute("select Name from Affiliations a")
all_affiliations = cursor.fetchall()
# university csv to aggregate the universities automatically based on the complete name in english
universities = pd.read_csv("world-universities.csv", names=['coutnry', 'university',"link"])["university"]
uni_count = {}
for uni in universities["name"]:
    uni_count[uni.lower()] = 0
for idx, uni in enumerate(uni_count):
    if idx % 250 == 0:
        print(idx)
    for affiliation in all_affiliations:
        affiliation = affiliation[0].lower()
        try:
            if re.search(f"(^|\s){uni}($|\s)",affiliation):
                uni_count[uni] += 1
        except:
            continue
non_zero_university = {k: v for k, v in uni_count.items() if v > 1}
non_zero_university_sorted = sorted(non_zero_university.items(), key=lambda x:x[1], reverse=True)
non_zero_university_sorted

FileNotFoundError: [Errno 2] No such file or directory: 'world-universities.csv'

In [ ]:
for university in non_zero_university_sorted:
    search_term = university[0]
    cursor.execute("select AffiliationID, Name from Affiliations where name like %s", [f"%{search_term}%"])
    all_matching_affiliations = cursor.fetchall()
    idx = get_affiliation_id(search_term)
    insert_affiliations(idx, all_matching_affiliations)
    cnx.commit()

In [212]:
t = []
cursor.execute("select * from Affiliations where entity_name is not null")
items = cursor.fetchall()
for item in items:

    if "\n" in item[3]:
        correct = item[3].split("\n")[-1]
        cursor.execute("update Affiliations a set a.entity_name = %s where a.AffiliationID=%s",[correct,item[0]])

In [213]:
cnx.commit()

### Manual Aggregation

In [7]:
# Run this script to find affiliations that have a common regex pattern
search_term = "Ford"
cursor.execute("select AffiliationID, Name from Affiliations where Affiliations.entity_name regexp %s or name regexp %s", [f"{search_term}", f"{search_term}"])
all_matching_affiliations = cursor.fetchall()
print(len(all_matching_affiliations))
all_matching_affiliations

60


[(213, 'University of Oxford'),
 (262, 'Stanford University'),
 (1013, 'University of Bradford'),
 (1092, 'University of Bedfordshire'),
 (1150, 'Oxford Brookes University'),
 (2676, 'University of Salford'),
 (2707, 'Staffordshire University'),
 (3840, 'Ashford University'),
 (3845, 'Fordham University'),
 (4320, 'Guilford Technical Community College'),
 (6633, 'Aware, Inc., Bedford, MA'),
 (6898, 'iRobot Corporation, Bedford, MA'),
 (7368, 'University of Hertfordshire'),
 (7579, 'Oxford Brain Diagnostics'),
 (7692, 'The Oxford Educational Institutions'),
 (12458, 'e2v technologies, 106 Waterhouse Lane, CM1 2QU, Chelmsford, Essex'),
 (12689,
  'Sanford Consortium for Regenerative Medicine, La Jolla, California 92037'),
 (13864,
  'Present addresses: Cancer Research UK London Research Institute, Clare Hall Laboratories, South Mimms, Hertfordshire EN6 3LD, UK (B.S.S.) and National Cardiovascular Center Research Institute, Osaka'),
 (14309, 'Salford Systems, 92123, San Diego, CA'),
 (228

In [5]:
# If the affiliations fit choose a title to aggregate it into
title = "Alphabet Inc."

In [6]:
# aggregate it
if title: #23222
    to_filter = [50824, 36371]
    all_matching_affiliations = [x[0] for x in all_matching_affiliations]
    all_matching_affiliations = list(filter(lambda x:x not in to_filter, all_matching_affiliations))
    idx = get_affiliation_id(title)
    insert_affiliations(idx, all_matching_affiliations)
title = None

NameError: name 'get_affiliation_id' is not defined

In [1032]:
# The next two cells' purpose is to rename a specific entry
replacement = {3699: ["Chinese Academy of Sciences"]}

In [1033]:
for key, value in replacement.items():
    cursor.execute("select * from affiliatedTo a where a.affiliatedTo = %s", [key])
    selected_authors = cursor.fetchall()
    print(selected_authors)
    for added_affiliation in value:
        cursor.execute("SELECT * FROM Affiliations a WHERE a.Name=%s", [added_affiliation])
        entry = cursor.fetchone()
        if not entry:
            cursor.execute("INSERT INTO Affiliations (Name) VALUES (%s)", [added_affiliation])
            lastrowid = cursor.lastrowid
        else:
            lastrowid = entry[0]
        for selected_author in selected_authors:
            try:
                cursor.execute("INSERT INTO affiliatedTo VALUES (%s,%s)", [selected_author[0],lastrowid])
            except mysql.connector.IntegrityError:
                continue
    cursor.execute("delete from affiliatedTo a where a.affiliatedTo = %s", [key])
    cursor.execute("delete from Affiliations a where a.AffiliationID = %s", [key])
    cnx.commit()
replacement = None

[('120156945', 3699)]


In [6]:
cursor.execute("select a.entity_name from Affiliations a where entity_name != '' and entity_name != 'na' and entity_name != ' na' group by a.entity_name having count(*) > 1")
for grouped_value in cursor.fetchall():
    idx = get_affiliation_id(grouped_value[0])
    cursor.execute("select a.AffiliationID from Affiliations a where a.entity_name=%s", [grouped_value[0]])
    results = [x[0] for x in cursor.fetchall()]
    insert_affiliations(idx, results)


In [ ]:
# Find most common words in affiliations
def word_count(str):
    counts = dict()
    words = str.split()
    print(len(words))
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

In [866]:
cursor.execute("select * from Affiliations a")
all_affiliations = cursor.fetchall()
all_text = ""
for affiliation in all_affiliations:
    all_text += f" {affiliation[1]}"
all_text = all_text.replace(",", " ").replace("-", " ").replace(".", " ")
{k: v for k, v in sorted(word_count(all_text).items(), key=lambda item: item[1], reverse=True)}

# Add Country to each place

In [74]:
gmaps = googlemaps.Client(key='')

In [80]:
cursor.execute("select * from Affiliations a where a.Country is NULL")
all_affiliations = cursor.fetchall()
len(all_affiliations)

1882

In [82]:
for idx,aff in enumerate(all_affiliations):
    if idx % 10 == 0:
        print(idx)
    try:
        output = gmaps.find_place(input=f"{aff[1]}",input_type="textquery")
        if output["status"] == 'ZERO_RESULTS':
            cursor.execute("UPDATE Affiliations a SET a.Country = %s WHERE a.AffiliationID = %s", ["None", aff[0]])
            continue
        place_id = output["candidates"][0]["place_id"]
        addresses = gmaps.place(place_id, fields=["address_component"])
        for address in addresses["result"]["address_components"]:
            if "country" in address["types"]:
                cursor.execute("UPDATE Affiliations a SET a.Country = %s WHERE a.AffiliationID = %s", [address["long_name"], aff[0]])
                print(aff[1])
                print(address["long_name"])
    except googlemaps.exceptions.ApiError as e:
        print(e)
        cursor.execute("UPDATE Affiliations a SET a.Country = %s WHERE a.AffiliationID = %s", ["None", aff[0]])
        continue
    cnx.commit()

0
SH Consulting KK (at the time of development)
Germany
SH Consulting KK
Germany
10
Technical University of Munich
Germany
Augmented Vision research department, DFKI- German Research Center for Artificial Intelligence
Germany
RWTH Aachen University
Germany
University of Tübingen
Germany
University of Bonn
Germany
NavVis
Germany
Amazon
Germany
AI Platform, Kwai Inc.
Germany
EVS Broadcast Equipment
Germany
Leibniz University Hannover
Germany
20
Scape Technologies
Germany
University of Freiburg
Germany
NLPR
Germany
n/a
Germany
Inception Institute of AI
Germany
General Motors
Germany
Robert Bosch GmbH
Germany
Pripares GmbH
Germany
Ubimax GmbH
Germany
Heidelberg University
Germany
30
World Bank
Germany
Karlsruhe Institute of Technology
Germany
lin
Germany
MVTec Software GmbH
Germany
University of Konstanz
Germany
AWS
Germany
MILA
Germany
DarkMatter AI Research
Germany
Fortiss - Research Institute of the Free State of Bavaria
Germany
X Inc.
Germany
40
Arm Research
Germany
Brandenburg Klinik


# Add Gender to each Name

In [5]:
cursor.execute("Select a.AuthorID, a.Name from Authors a where Gender is NULL")
authors = cursor.fetchall()
len(authors)

68612

In [6]:
names_to_predict = []
ids_to_predict = []

for idx,author in enumerate(authors):

    if not author[1]:
        continue
    name = author[1].split(" ")

    first_name = name[0]
    if "." in first_name:
        if len(name) > 2:
            first_name = name[2]
        if len(name) <= 2 or "." in first_name:
            continue
    names_to_predict.append(first_name)
    ids_to_predict.append(author[0])

    if len(names_to_predict) == 10:
        gender_list = requests.get(f"https://api.genderize.io?name[]={names_to_predict[0]}&name[]={names_to_predict[1]}&name[]={names_to_predict[2]}&name[]={names_to_predict[3]}&name[]={names_to_predict[4]}&name[]={names_to_predict[5]}&name[]={names_to_predict[6]}&name[]={names_to_predict[7]}&name[]={names_to_predict[8]}&name[]={names_to_predict[9]}&apikey=d010e94326a0e0d331dcf6106a17bddd").json() #&apikey=6ed07fffb407dbd3f394d2c8afbc1fcb
        for gender_id,gender_resp in enumerate(gender_list):
            isFemale = 0
            if gender_resp["gender"] == "female":
                isFemale = 1
            if gender_resp["gender"] == "None":
                isFemale = 2
            cursor.execute("UPDATE Authors a SET a.Gender = %s WHERE a.AuthorID = %s", [isFemale, ids_to_predict[gender_id]])
        cnx.commit()
        names_to_predict = []
        ids_to_predict = []
    if idx % 10 == 0:
        print(idx)

6310
6320
6330
6340
6350
6360
6370
6380
6390
6400
6410
6420
6430
6440
6450
6460
6470
6480
6490
6500
6510
6520
6530
6540
6550
6560
6570
6580
6590
6600
6610
6620
6630
6640
6650
6660
6670
6680
6690
6700
6710
6720
6730
6740
6750
6760
6770
6780
6790
6800
6810
6820
6830
6840
6850
6860
6870
6880
6890
6900
6910
6920
6930
6940
6950
6960
6970
6980
6990
7000
7010
7020
7030
7040
7050
7060
7070
7080
7090
7100
7110
7120
7130
7140
7150
7160
7170
7180
7190
7200
7210
7220
7230
7240
7250
7260
7270
7280
7290
7300
7310
7320
7330
7340
7350
7360
7370
7380
7390
7400
7410
7420
7430
7440
7450
7460
7470
7480
7490
7500
7510
7520
7530
7540
7550
7560
7570
7580
7590
7600
7610
7620
7630
7640
7650
7660
7670
7680
7690
7700
7710
7720
7730
7740
7750
7760
7770
7780
7790
7800
7810
7820
7830
7840
7850
7860
7870
7880
7890
7900
7910
7920
7930
7940
7950
7960
7970
7980
7990
8000
8010
8020
8030
8040
8050
8060
8070
8080
8090
8100
8110
8120
8130
8140
8150
8160
8170
8180
8190
8200
8210
8220
8230
8240
8250
8260
8270
8280
8290
8300


In [21]:
cnx.commit()